# Machine Learning - Apprentissage par renforcement
## Qu'est-ce que l'apprentissage par renforcement ?

L'apprentissage par renforcement (AR) est une méthode clé en intelligence artificielle et en apprentissage automatique, distincte des approches supervisées et non supervisées. Dans ce cadre, un agent apprend à prendre des décisions en interagissant avec un environnement, recevant des récompenses ou des pénalités selon ses actions. L'objectif principal est de maximiser les récompenses cumulatives au fil du temps[1][2][4].

### Principe de fonctionnement

L'AR repose sur le concept de **processus de décision markovien**, où l'agent doit choisir des actions basées sur l'état actuel de l'environnement. Chaque action entraîne une transition vers un nouvel état et génère une récompense. Le défi consiste à attribuer correctement les récompenses aux actions qui ont conduit à un résultat positif, un phénomène connu sous le nom de **problème d'attribution de crédit**[2][3].

### Composantes clés

Les éléments fondamentaux de l'apprentissage par renforcement incluent :

- **Agent** : L'entité qui prend des décisions.
- **Environnement** : Le système avec lequel l'agent interagit.
- **Politique** : Une stratégie définissant l'action à prendre dans chaque état.
- **Récompense** : Un signal qui évalue la qualité de l'action effectuée.
- **Valeur** : Une estimation de la récompense future attendue[2][4].

## Applications de l'apprentissage par renforcement

L'apprentissage par renforcement a trouvé des applications variées dans plusieurs domaines :

- **Robotique** : Utilisé pour enseigner aux robots comment manipuler des objets ou naviguer dans des environnements complexes[1][3].
- **Jeux vidéo** : Des systèmes comme AlphaGo ont démontré la puissance de l'AR en battant des champions humains dans des jeux stratégiques[5].
- **Gestion intelligente des ressources** : Optimisation de la consommation d'énergie dans les centres de données ou contrôle automatisé des feux tricolores[3][4].
- **Finance et santé** : Application dans la prise de décisions complexes, où les agents apprennent à s'adapter à des environnements dynamiques[2][4].

## Défis et perspectives

Malgré ses succès, l'apprentissage par renforcement présente plusieurs défis :

- **Simulation réaliste** : Créer un environnement d'apprentissage qui reflète fidèlement le monde réel est crucial, surtout pour des applications telles que les véhicules autonomes[4].
- **Évolutivité et robustesse** : Les algorithmes doivent être capables de gérer la variabilité et l'imprévisibilité des environnements réels[2][5].
- **Dépendance aux données** : Bien que l'AR puisse réduire le besoin de grandes quantités de données étiquetées, il nécessite toujours une quantité suffisante d'interactions pour apprendre efficacement[3].


En conclusion, l'apprentissage par renforcement est une méthode puissante qui continue d'évoluer, avec un potentiel significatif pour transformer divers secteurs grâce à ses capacités d'adaptation et d'optimisation.
![image](https://gymnasium.farama.org/_images/AE_loop_dark.png)
References:
* [1] https://fr.wikipedia.org/wiki/Apprentissage_par_renforcement
* [2] https://www.ovhcloud.com/fr/learn/what-is-reinforcement-learning/
* [3] https://larevueia.fr/apprentissage-par-renforcement/
* [4] https://www.lebigdata.fr/reinforcement-learning-definition
* [5] https://datascientest.com/reinforcement-learning
* [6] https://fc.sorbonne-universite.fr/nos-offres/apprentissage-par-renforcement-intelligence-artificielle/
* [7] https://aws.amazon.com/fr/what-is/reinforcement-learning/
* [8] https://loud-technology.com/programmation/definitions/apprentissage-par-renforcement/

In [2]:
import gymnasium as gym
import numpy as np

env = gym.make("LunarLander-v2", render_mode="human")

# Hyper-parametres

alpha = 0.1
gamma = 0.99
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01
episodes = 1000
max_steps = 1000

# print(env.observation_space.shape)

# Créer des bins (intervales) pour chaque dimension de l'état afin de discrétiser l'espace d'observation continu
state_bins = [np.linspace(-1.0, 1.0, 10) for _ in range(env.observation_space.shape[0])]

# l'ajout de  1 permet de couvrir les valeurs hors limites
n_bins = tuple(len(bins) + 1 for bins in state_bins)

# Initialisation de la table q avec les bonnes dimensions
q_table = np.zeros(n_bins + (env.action_space.n,))

def discretize_state(state):
    return tuple(np.digitize(state[i], state_bins[i]) for i in range(len(state)))

# print(state_bins)
print(q_table)


# ____________________________________________________

for episode in range(episodes):
    # Réinitialisation de l'environnement au début de chaque épisode
    state, _ = env.reset(seed=19)
    state = discretize_state(state)
    total_reward = 0

    for step in range(max_steps):
        # Politique epsilon-greedy pour choisir l'action (exploration vs exploitation)
        if np.random.uniform(0, 1) < epsilon:
            # Exploration : choisir une action aléatoire
            action = env.action_space.sample()
        else:
            # Exploitation : choisir l'action avec la valeur Q maximale pour l'état actuel
            action = np.argmax(q_table[state])

        # Appliquer l'action choisie et récupérer la nouvelle observation
        next_state, reward, terminated, truncated, _ = env.step(action)
        next_state = discretize_state(next_state)

        # Obtenir la valeur Q actuelle et la valeur Q maximale de l'état suivant
        old_value = q_table[state][action]  # Valeur Q actuelle
        next_max = np.max(q_table[next_state])  # Valeur Q max de l'état S'

        # Mise à jour de la table Q avec la formule de Bellman
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state][action] = new_value

        # Passer à l'état suivant
        state = next_state
        total_reward += reward

        # Si l'épisode se termine (par réussite ou échec), sortir de la boucle
        if terminated or truncated:
            break

    # Réduction d'epsilon pour diminuer progressivement l'exploration
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    # Affichage du total de récompenses à la fin de chaque épisode
    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

# Fermeture de l'environnement
env.close()


Hello World!
